In [ ]:
import numpy as np
import optuna
import json
import matplotlib.pyplot as plt
from bcmix import *

In [ ]:
ACTION_RANGE = (-5.0, 5.0)
N_TRIALS = 100
DATA_LEN = 8

In [ ]:
np.random.seed(16)

# prior
canonical_0 = np.array([[0.0], [0.0]])
precision_0 = np.array([[1.0, 0.0], [0.0, 1.0]])
covm_0 = np.linalg.inv(precision_0)
mean_0 = covm_0 @ canonical_0

# true value
#alpha, beta = np.random.multivariate_normal(mean_0.flatten(), covm_0)
alpha, beta = 1.8, -2.4
print(alpha, beta, myopic(canonical_0, precision_0))

### Rollout

In [ ]:
def objective(trial):
    a = trial.suggest_float('a', ACTION_RANGE[0], ACTION_RANGE[1])
    q = q_myopic_without_change(canonical_i, precision_i, a, alpha, beta)
    return q

In [ ]:
np.random.seed(None)

canonical_i, precision_i = canonical_0, precision_0
states, actions, rewards, qvalues = [], [], [], []

In [ ]:
for _ in range(10):
    canonical_i, precision_i = canonical_0, precision_0
    states, actions, rewards, qvalues = [], [], [], []
    for i in range(DATA_LEN):
        # current state
        covm_i = np.linalg.inv(precision_i)
        mean_i = covm_i @ canonical_i
        states.append((mean_i, covm_i))
        # select action
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=N_TRIALS)


        
        a = study.best_trial.params['a']
        actions.append(a)
        qvalues.append(study.best_trial.value)
        # update state
        y = env_response(a, alpha, beta)[0]
        rewards.append(reward(a, y))
        canonical_i, precision_i = update_without_change(canonical_i, precision_i, a, y)
"""
    with open("simulations/rollout_without_change.json", 'a') as fout:
        json.dump(
            {"states": [s[0].flatten().tolist() + s[1].flatten().tolist() for s in states], "actions": actions, "qvalues": qvalues}, fout
        )
        fout.write('\n')
"""

In [ ]:
results = []
with open("simulations/rollout_without_change.json") as fin:
    for line in fin:
        results.append(json.loads(line))

In [ ]:
covm_i

In [ ]:
study.trials[0]

In [ ]:
result = results[0]
plt.plot(result["qvalues"])

In [ ]:
plt.plot([_[0] for _ in result["states"]])

In [ ]:
plt.plot([_[1] for _ in result["states"]])

### Simulate observed data

In [ ]:
xs = np.random.uniform(ACTION_RANGE[0], ACTION_RANGE[1], DATA_LEN)
ys = np.array([env_response(x, alpha, beta)[0] for x in xs])